# Animations

To create animations with Plopp, we make use of the [node mechanism](custom-interfaces.ipynb)
and leverage [Matplotlib's support for animations](https://matplotlib.org/stable/users/explain/animations/animations.html).

## 1D animations

In [ ]:
import plopp as pp
from plopp.data.examples import three_bands
import matplotlib.animation as animation

We load the data and display it in a figure:

In [ ]:
da = three_bands()
da.plot(norm="log")

We now create an animation where we are scanning the vertical `y` direction of the data,
making a plot of each slice in each step of the animation.

On every call to the `update` function passed to Matplotlib's `FuncAnimation`,
we update the index node and notify the children nodes of the change.
This updates the figure at every step.

Finally, we save the animation to a `.gif` file.
See [here](https://matplotlib.org/stable/users/explain/animations/animations.html#saving-animations)
for the possible formats animations can be saved in.

In [ ]:
data_node = pp.Node(da)
index_node = pp.Node(0)
# Here, 'y' is the dimension you want to slice
slice_node = pp.Node(lambda da, ind: da["y", ind], da=data_node, ind=index_node)

f = pp.linefigure(slice_node, vmax=200)


def update(frame):
    index_node.func = lambda: frame
    index_node.notify_children("")


ani = animation.FuncAnimation(fig=f.fig, func=update, frames=200, interval=100)
ani.save(filename="animation_1d.gif", writer="pillow")

We can now load the GIF image and display it:

In [ ]:
import ipywidgets as ipw

ipw.Image(value=open("animation_1d.gif", "rb").read(), format="gif")

## 2D animations

We can create a similar animation on 2d images,
where instead of slicing the vertical dimension,
we collect more and more signal as time progresses.

Every time the `update` function is called,
new data is created and added to the current data.

In [ ]:
da = three_bands(npeaks=20, per_peak=500, spread=30.0)
data_node = pp.Node(da)

f = pp.imagefigure(data_node, cbar=True, vmin=30, vmax=400, norm="log")


def update(frame):
    # Make new data and add it to the current data.
    # The current data is obtained by calling the `data_node()`
    new = three_bands(npeaks=20, per_peak=500, spread=30.0) + data_node()
    data_node.func = lambda: new
    data_node.notify_children("")


ani = animation.FuncAnimation(fig=f.fig, func=update, frames=80, interval=100)
ani.save(filename="animation_2d.gif", writer="pillow")

We now display the resulting animation:

In [ ]:
ipw.Image(value=open("animation_2d.gif", "rb").read(), format="gif")

## 3D animations

Animations on 3D plots are currently not supported.

The best way to achieve this would be to record a screencast while interacting with a 3D scene.

In [ ]:
# This cell is used to generate the thumbnail for the docs.
# It is hidden from the online documentation.
import shutil

shutil.copyfile("animation_2d.gif", "../_static/customization/animation_2d.gif")